In [1]:
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [2]:
#les données

results = pd.read_csv(r"data/results.csv", sep=',')
races = pd.read_csv(r"data/races.csv", sep=',')
circuits = pd.read_csv(r"data/circuits.csv", sep=',')
constructors = pd.read_csv(r"data/constructor_standings.csv", sep=',')

df1 = pd.merge(results[['driverId', 'constructorId', 'raceId', 'positionOrder']], races[['raceId', 'year', 'circuitId']], on = 'raceId')
df2 = pd.merge(df1, constructors[['raceId', 'constructorId', 'position', 'points']], on = ['raceId', 'constructorId'])
df3 = pd.merge(df2, circuits[['circuitId','country']], on = 'circuitId')
df3['winner'] = df3['positionOrder'].apply(lambda x: 1 if x==1 else 0)
df4 = df3.drop(['circuitId', 'constructorId', 'positionOrder'], axis = 1)#.set_index(['raceId', 'driverId'])

label_encoder = LabelEncoder()
df4.loc[:, 'country'] = label_encoder.fit_transform(df3['country'])
df4.loc[:,"position**2"] = df4["position"] ** 2
df4.loc[:,"points**2"] = df4["points"] ** 2
df4 = df4.rename(columns={"points" : 'pointsequipe'})

In [3]:
print(df4)
print(df4.columns)
df4 = df4.set_index(['raceId','driverId'])

       driverId  raceId  year  position  pointsequipe country  winner  \
0             1      18  2008         1          14.0       1       1   
1             2      18  2008         3           8.0       1       0   
2             3      18  2008         2           9.0       1       0   
3             4      18  2008         4           5.0       1       0   
4             5      18  2008         1          14.0       1       0   
...         ...     ...   ...       ...           ...     ...     ...   
23968       854    1096  2022         8          37.0      30       0   
23969       825    1096  2022         8          37.0      30       0   
23970         1    1096  2022         3         515.0      30       0   
23971       849    1096  2022        10           8.0      30       0   
23972         4    1096  2022         4         173.0      30       0   

       position**2  points**2  
0                1      196.0  
1                9       64.0  
2                4       81

In [4]:
df4['year'] = df4['year'].astype(int)
df4['position'] = df4['position'].astype(int)
df4['pointsequipe'] = df4['pointsequipe'].astype(int)
df4['country'] = df4['country'].astype(int)
df4['winner'] = df4['winner'].astype(int)
df4['position**2'] = df4['position**2'].astype(int)
df4['points**2'] = df4['points**2'].astype(int)


In [5]:
df4_pre= df4[['year', 'position', 'pointsequipe', 'country','winner', 'position**2', 'points**2']]
df4_train = df4_pre[df4_pre['year']<=2020]
df4_test = df4_pre[df4_pre['year']==2021]

X_df4_train = df4_train.drop(columns='winner')
Y_df4_train = df4_train[['winner']]

X_df4_test = df4_test.drop(columns='winner')
Y_df4_test = df4_test[['winner']]

In [6]:
XBoost = XGBClassifier(random_state = 23,n_estimators=25,learning_rate=0.1,objective='reg:squaredlogerror',eval_metric="logloss")
XBoost.fit(X_df4_train,Y_df4_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=25,
              n_jobs=None, num_parallel_tree=None,
              objective='reg:squaredlogerror', ...)

In [7]:
XBoost.score(X_df4_train,Y_df4_train)
#XBoost.predict_proba(X_df4_test)

0.9580825358333694

In [8]:
def fromPROBA_to_WinnerAndPodium(X_dateSet, Y_dataSet, Model):
    y_pred = Y_dataSet
    probabilite = Model.predict_proba(X_dateSet)
    probabilite = pd.DataFrame(probabilite)
    probabilite = probabilite.drop(columns=(0))
    probabilite = np.array(probabilite)
    y_pred.loc[:,'PREpositionOrder'] = probabilite
    
    Taille = y_pred.index
    raceID = Taille.get_level_values('raceId').unique()
    PodiumOr0 = []
    Top1 = []
    for RACE in raceID:
        driverID = y_pred["PREpositionOrder"][RACE].index.get_level_values('driverId')
        array = np.array(y_pred["PREpositionOrder"][RACE])
        ranks = array.argsort()
        nb = len(driverID)
        ranks = nb - ranks.argsort()
        count = 0
        for DRIVER in driverID:
            rang = ranks[count]
            if rang <= 3:
                PodiumOr0.append(ranks[count])
                if rang == 1:
                    Top1.append(1)
                else:
                    Top1.append(0)
            else:
                PodiumOr0.append(0)
                Top1.append(0)
            count += 1 
        count = 0

    y_pred.loc[:,'winner'] = Top1
    y_pred.loc[:,'positionOrder'] = PodiumOr0
    fin = y_pred
    return fin

In [9]:
Y_df4_test = fromPROBA_to_WinnerAndPodium(X_df4_test,Y_df4_test, XBoost)

C:\Users\luang\AppData\Local\Temp\ipykernel_13808\2171731140.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_pred.loc[:,'PREpositionOrder'] = probabilite
C:\Users\luang\AppData\Local\Temp\ipykernel_13808\2171731140.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_pred.loc[:,'positionOrder'] = PodiumOr0


In [10]:
#Y_df4_test = Y_df4_test.drop(columns='PREpositionOrder')
Y_df4_test.to_csv("modelXGboost.csv")

result : 64.43